In [3]:
from efficient_apriori import apriori
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
from surprise import Dataset
from surprise import Reader
from surprise.model_selection.split import train_test_split
from surprise.model_selection import cross_validate, GridSearchCV
import numpy as np
import io
from surprise import SlopeOne, NMF,NormalPredictor, CoClustering
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore,KNNBaseline
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import SVD
from surprise import accuracy
import matplotlib.pyplot as plt

In [127]:
#loads metadata
import json
data=[]
with open('/Users/apoorvabanubakode/Downloads/meta_Musical_Instruments.json') as f:
    for l in f:
        data.append(json.loads(l.strip()))
AppliancesmetaDf=pd.DataFrame(data)

In [129]:
#clean dataset by filtering df where unformatted rows present  
df3 = AppliancesmetaDf.fillna('')
df4 = df3[df3.title.str.contains('getTime')] 
df5 = df3[~df3.title.str.contains('getTime')] 
print(len(df4))
print(len(df5))
AppliancesmetaDf_cleaned=df5.copy()


1907
118493


In [130]:
#loads reviews and rating data 
dataAppliances=[]
with open('/Users/apoorvabanubakode/Downloads/Musical_Instruments.json') as f:
    for l in f:
        dataAppliances.append(json.loads(l.strip()))

AppliancesDf=pd.DataFrame(dataAppliances)



In [131]:
AppliancesDf.head()

,asin,image,overall,reviewText,reviewTime,reviewerID,reviewerName,style,summary,unixReviewTime,verified,vote
0,0470536454,NaN,5.0,Crocheting for Dummies by Karen Manthey & Susa...,"08 9, 2004",AXHY24HWOF184,Bendy,{'Format:': ' Paperback'},Terrific Book for Learning the Art of Crochet,1092009600,False,90
1,0470536454,NaN,4.0,Very helpful...,"04 6, 2017",A29OWR79AM796H,Amazon Customer,{'Format:': ' Hardcover'},Four Stars,1491436800,True,2
2,0470536454,NaN,5.0,EASY TO UNDERSTAND AND A PROMPT SERVICE TOO,"03 14, 2017",AUPWU27A7X5F6,Amazon Customer,{'Format:': ' Paperback'},Five Stars,1489449600,True,NaN
3,0470536454,NaN,4.0,My girlfriend use quite often,"02 14, 2017",A1N69A47D4JO6K,Christopher Burnett,{'Format:': ' Paperback'},Four Stars,1487030400,True,NaN
4,0470536454,NaN,5.0,Arrived as described. Very happy.,"01 29, 2017",AHTIQUMVCGBFJ,Amazon Customer,{'Format:': ' Paperback'},Very happy.,1485648000,True,NaN


In [132]:
AppliancesmetaDf_cleaned.head()

,also_buy,also_view,asin,brand,category,date,description,details,feature,fit,image,main_cat,price,rank,similar_item,tech1,tech2,title
0,"[B00NP8GYVS, B00NP80XMO, B00NP8M098]",,0000989983,WADSUWAN SHOP,"[Musical Instruments, Drums & Percussion, Hand...","December 2, 2013",[Cricket Rubbing the spine with the wooden sti...,,"[Wood percussion, Owl whistle*, Includes woode...",,[https://images-na.ssl-images-amazon.com/image...,Musical Instruments,,">#141,729 in Musical Instruments (See Top 100 ...",,,,Wooden Percussion 2 Piece Set of 3 Inch Cricke...
1,"[B00NP8GYVS, B00NP80XMO, B01MY48HK5, B00AZZ1AJ...",,0000098906,WADSUWAN SHOP,"[Musical Instruments, Drums & Percussion, Hand...","December 2, 2013",[Frog - Rubbing its spine with the wooden stic...,,"[Wood percussion, Small 3 inches, Creates orig...",,[https://images-na.ssl-images-amazon.com/image...,Musical Instruments,$0.91,">#1,622 in Musical Instruments (See Top 100 in...",,,,"Wooden Percussion 3 Piece Set Frog, Cricket an..."
2,,,0041291905,Hal Leonard,"[Musical Instruments, Instrument Accessories, ...","May 10, 2011",[Vivaldi's famous set of four violin concertos...,,"[., ., .]",,[https://images-na.ssl-images-amazon.com/image...,Musical Instruments,$62.93,">#330,653 in Musical Instruments (See Top 100 ...",,,,Hal Leonard Vivaldi Four Seasons for Piano (Or...
3,"[0486266842, 0793507669, 0393008789, 142341280...",,0060015500,Boosey &amp; Hawkes,,"May 23, 2007","[The Turn of the Screw (op. 54) vocal score, p...",,,,[https://images-na.ssl-images-amazon.com/image...,Musical Instruments,$107.79,"[>#86,354 in Musical Instruments (See Top 100 ...",,,,The Turn of the Screw (vocal score)
4,,,0193757710,,,"February 8, 2013",,,,,[https://images-na.ssl-images-amazon.com/image...,Musical Instruments,,"[>#482,025 in Musical Instruments (See Top 100...",,,,Suite for Organ (including the Trumpet Volunta...


In [133]:
subset_ratings=AppliancesDf[['asin','reviewTime','reviewerID','verified','overall','reviewText']]
subset_ratings.shape

(1512530, 6)

In [134]:
#filter non verified users
subset_ratings=subset_ratings[subset_ratings.verified==True]
subset_ratings['reviewDate']=pd.to_datetime(subset_ratings['reviewTime'])
subset_ratings.head()

,asin,reviewTime,reviewerID,verified,overall,reviewText,reviewDate
1,0470536454,"04 6, 2017",A29OWR79AM796H,True,4.0,Very helpful...,2017-04-06
2,0470536454,"03 14, 2017",AUPWU27A7X5F6,True,5.0,EASY TO UNDERSTAND AND A PROMPT SERVICE TOO,2017-03-14
3,0470536454,"02 14, 2017",A1N69A47D4JO6K,True,4.0,My girlfriend use quite often,2017-02-14
4,0470536454,"01 29, 2017",AHTIQUMVCGBFJ,True,5.0,Arrived as described. Very happy.,2017-01-29
5,0470536454,"01 4, 2017",A1J8LQ7HVLR9GU,True,5.0,Love the Dummies Series. Never fails.,2017-01-04


In [135]:
#check size of dataset with various time frames
subset_ratings[subset_ratings.reviewDate>'2018-01-01'].shape

(124680, 7)

In [136]:
#experiment with various datasets generated by effective data filtering : 2016,2017,2018
subset_ratings=subset_ratings[subset_ratings.reviewDate>'2018-01-01']

In [137]:
checkdf=subset_ratings[subset_ratings.duplicated(subset=['asin','reviewerID','reviewDate'],keep=False)].sort_values(['asin','reviewerID','reviewDate'])
checkdf.shape

subset_ratings=subset_ratings.drop_duplicates(subset=['asin','reviewerID','reviewDate'],keep='last')
subset_ratings.shape

(124207, 7)

In [138]:
checkdf=subset_ratings[subset_ratings.duplicated(subset=['asin','reviewerID','reviewDate'],keep=False)].sort_values(['asin','reviewerID','reviewDate'])
checkdf.shape

subset_ratings=subset_ratings.drop_duplicates(subset=['asin','reviewerID','reviewDate'],keep='last')
subset_ratings.shape

(124207, 7)

In [139]:
subset_ratings=subset_ratings[['reviewerID','asin','overall']]
#currently for svd /knn models only consider product id ; reviewer id and ratings
#later when top 10 findings then select according to dates and categoty and other details of the asin id 

In [140]:
no_of_products_reviewed=subset_ratings.asin.value_counts()
no_of_products_reviewed.describe([.95,.97,.99])
# we can take products who have more than 50 reviews to start with ; but lets try iwith actual given products

count    26055.000000
mean         4.767108
std         16.309818
min          1.000000
50%          1.000000
95%         16.000000
97%         24.000000
99%         60.460000
max        481.000000
Name: asin, dtype: float64

In [141]:
len(no_of_products_reviewed) # unique product count 

26055

In [142]:
review_per_user_counts=subset_ratings.reviewerID.value_counts()
review_per_user_counts.describe([.95,.97,.99]) # unique user count 

count    94543.000000
mean         1.313762
std          1.038768
min          1.000000
50%          1.000000
95%          3.000000
97%          4.000000
99%          6.000000
max         57.000000
Name: reviewerID, dtype: float64

In [143]:
atemp=subset_ratings.reviewerID.value_counts()
atemp[atemp>=3].shape

(5864,)

In [144]:
filtered_users=review_per_user_counts[review_per_user_counts>=3].reset_index()

In [145]:
filtered_users=filtered_users['index'].tolist()

In [146]:
subset_backup=subset_ratings.copy()

In [147]:
subset_ratings=subset_ratings[subset_ratings.reviewerID.isin(filtered_users)]

In [148]:
musical=subset_ratings

In [149]:
musical=musical[['reviewerID','asin','overall']]
#cast into subset ratings
subset_ratings=musical

In [150]:
#filter users with morre than 10 reviews 
val=subset_ratings.reviewerID.value_counts()
val=val.reset_index()
owners_10ormore=val[val.reviewerID>=10]
owners_10ormore=owners_10ormore.drop_duplicates(subset=['index'])
owners_10ormore=list(owners_10ormore['index'])
len(owners_10ormore)

200

In [151]:
subset_ratings=subset_ratings[subset_ratings.reviewerID.isin(owners_10ormore)]
subset_ratings.shape
#earlier shape -(2454435, 3)

(2774, 3)

In [152]:
#create a holdout dataset for these 200 users for evaulation 
subset_ratings_train=pd.DataFrame()
subset_ratings_test=pd.DataFrame()
for owner in owners_10ormore[:]:
    ownersubset=subset_ratings[subset_ratings.reviewerID==owner]
    splitpoint=int(ownersubset.shape[0]*0.8)
    #80% appended to training df 
    #20% appended to testing df
    subset_ratings_train=pd.concat([subset_ratings_train,ownersubset.iloc[:splitpoint,:]],axis=0)
    subset_ratings_test=pd.concat([subset_ratings_test,ownersubset.iloc[splitpoint:,:]],axis=0)
    
    #print(subset_ratings_train.shape)
    #print(subset_ratings_test.shape)

In [153]:
subset_ratings_train['asin'].nunique()

1530

In [154]:
a=subset_ratings_train[['asin']]
#print(a.shape)
a=a.drop_duplicates()
#print(a.shape)
all_items_train=list(a.loc[:,'asin'])
#print(all_items_train)
ar=subset_ratings_train[['reviewerID']]
print(ar.shape)
ar=ar.drop_duplicates()
print(ar.shape)
all_users_train=list(ar.loc[:,'reviewerID'])

(2149, 1)
(200, 1)


In [155]:
import random

#choose 30 most frequent buyers for manual evaulation 
maxrevs=subset_ratings_train.reviewerID.value_counts().reset_index()
maxrevs_id=list(maxrevs.iloc[:30,0])
maxrevs_id
sampling_users = maxrevs_id

In [156]:
sampled_itemdf=pd.DataFrame(all_items_train,columns=['asin'])
sampled_itemdf.head()
sampling_usersdf=pd.DataFrame(sampling_users,columns=['reviewerID'])
sampling_usersdf.head()

,reviewerID
0,AO21QAFMIK124
1,A3EATLL9FW7DJL
2,AQDUE9PHA7QRP
3,A215GPS2IFRI7G
4,A7BTFA87UO4TZ


In [157]:
import itertools
result= list(itertools.product( sampling_users,all_items_train))

In [158]:
resultdf=pd.DataFrame(result,columns=['reviewerID','asin'])
resultdf.head()

,reviewerID,asin
0,AO21QAFMIK124,B0002DV6TO
1,AO21QAFMIK124,B0002E3CK4
2,AO21QAFMIK124,B0002F76QY
3,AO21QAFMIK124,B0002H03YY
4,AO21QAFMIK124,B0002H05BA


In [159]:
resultdf.shape
#this is test set 
resultdf['overall']=0
resultdf.head()

,reviewerID,asin,overall
0,AO21QAFMIK124,B0002DV6TO,0
1,AO21QAFMIK124,B0002E3CK4,0
2,AO21QAFMIK124,B0002F76QY,0
3,AO21QAFMIK124,B0002H03YY,0
4,AO21QAFMIK124,B0002H05BA,0


In [160]:
subset_ratings_train.asin.nunique()

1530

In [161]:
unique_item=subset_ratings_train.asin.unique()

In [163]:
AppliancesmetaDf_cleaned.shape

(118493, 18)

In [164]:
aprioridf=AppliancesmetaDf_cleaned.copy()

In [165]:
#filter rows where no alsobuy products present
aprioridf=aprioridf[aprioridf['also_buy']!='']
aprioridf.shape

(17125, 18)

In [166]:
aprioridf.head()

,also_buy,also_view,asin,brand,category,date,description,details,feature,fit,image,main_cat,price,rank,similar_item,tech1,tech2,title
0,"[B00NP8GYVS, B00NP80XMO, B00NP8M098]",,0000989983,WADSUWAN SHOP,"[Musical Instruments, Drums & Percussion, Hand...","December 2, 2013",[Cricket Rubbing the spine with the wooden sti...,,"[Wood percussion, Owl whistle*, Includes woode...",,[https://images-na.ssl-images-amazon.com/image...,Musical Instruments,,">#141,729 in Musical Instruments (See Top 100 ...",,,,Wooden Percussion 2 Piece Set of 3 Inch Cricke...
1,"[B00NP8GYVS, B00NP80XMO, B01MY48HK5, B00AZZ1AJ...",,0000098906,WADSUWAN SHOP,"[Musical Instruments, Drums & Percussion, Hand...","December 2, 2013",[Frog - Rubbing its spine with the wooden stic...,,"[Wood percussion, Small 3 inches, Creates orig...",,[https://images-na.ssl-images-amazon.com/image...,Musical Instruments,$0.91,">#1,622 in Musical Instruments (See Top 100 in...",,,,"Wooden Percussion 3 Piece Set Frog, Cricket an..."
3,"[0486266842, 0793507669, 0393008789, 142341280...",,0060015500,Boosey &amp; Hawkes,,"May 23, 2007","[The Turn of the Screw (op. 54) vocal score, p...",,,,[https://images-na.ssl-images-amazon.com/image...,Musical Instruments,$107.79,"[>#86,354 in Musical Instruments (See Top 100 ...",,,,The Turn of the Screw (vocal score)
5,"[1118661516, 0470045558, 156477998X, B00QF75NF...","[1119287111, 156477998X, 0470045558, 190870724...",0470536454,Wiley Publishing,"[Musical Instruments, Instrument Accessories, ...",,"[, An updated classic covering the latest tech...",,[Dimensions: 9.2 in. h x 7.46 in. w x 0.78 in....,,[https://images-na.ssl-images-amazon.com/image...,"Arts, Crafts & Sewing",,">#76,268 in Arts, Crafts & Sewing (See Top 100...",,,,Wiley Publishers Crocheting For Dummies Revised
11,"[B074XN2B21, B078S2GVQT, B01G2DLWZU, 149500914...",,0692218645,Troubadour Music Company,"[Musical Instruments, Instrument Accessories, ...","October 27, 2014",[The Ukulele Chord Changer fits soprano ukulel...,,"[Chords include C7, C, G7, G, A7, F, D7, and D...",,[https://images-na.ssl-images-amazon.com/image...,Musical Instruments,,">#26,851 in Musical Instruments (See Top 100 i...","[{'asin': '', 'href': '', 'name': 'Ukulele Cho...",,,Ukulele Chord Changer and Songbook Set


In [167]:
copydf=aprioridf.copy()

In [168]:
def appenfunc(oldlist,asin):
    #print(asin)
    oldlist.append(asin)
    
    return oldlist

In [ ]:
# append current product to also bought column to have a list of items bought together 

In [170]:
aprioridf['also_buy_appended']=aprioridf.apply(lambda x: appenfunc(x.also_buy,x.asin),axis=1)
aprioridf.head()

,also_buy,also_view,asin,brand,category,date,description,details,feature,fit,image,main_cat,price,rank,similar_item,tech1,tech2,title,also_buy_appended
0,"[B00NP8GYVS, B00NP80XMO, B00NP8M098, 0000989983]",,0000989983,WADSUWAN SHOP,"[Musical Instruments, Drums & Percussion, Hand...","December 2, 2013",[Cricket Rubbing the spine with the wooden sti...,,"[Wood percussion, Owl whistle*, Includes woode...",,[https://images-na.ssl-images-amazon.com/image...,Musical Instruments,,">#141,729 in Musical Instruments (See Top 100 ...",,,,Wooden Percussion 2 Piece Set of 3 Inch Cricke...,"[B00NP8GYVS, B00NP80XMO, B00NP8M098, 0000989983]"
1,"[B00NP8GYVS, B00NP80XMO, B01MY48HK5, B00AZZ1AJ...",,0000098906,WADSUWAN SHOP,"[Musical Instruments, Drums & Percussion, Hand...","December 2, 2013",[Frog - Rubbing its spine with the wooden stic...,,"[Wood percussion, Small 3 inches, Creates orig...",,[https://images-na.ssl-images-amazon.com/image...,Musical Instruments,$0.91,">#1,622 in Musical Instruments (See Top 100 in...",,,,"Wooden Percussion 3 Piece Set Frog, Cricket an...","[B00NP8GYVS, B00NP80XMO, B01MY48HK5, B00AZZ1AJ..."
3,"[0486266842, 0793507669, 0393008789, 142341280...",,0060015500,Boosey &amp; Hawkes,,"May 23, 2007","[The Turn of the Screw (op. 54) vocal score, p...",,,,[https://images-na.ssl-images-amazon.com/image...,Musical Instruments,$107.79,"[>#86,354 in Musical Instruments (See Top 100 ...",,,,The Turn of the Screw (vocal score),"[0486266842, 0793507669, 0393008789, 142341280..."
5,"[1118661516, 0470045558, 156477998X, B00QF75NF...","[1119287111, 156477998X, 0470045558, 190870724...",0470536454,Wiley Publishing,"[Musical Instruments, Instrument Accessories, ...",,"[, An updated classic covering the latest tech...",,[Dimensions: 9.2 in. h x 7.46 in. w x 0.78 in....,,[https://images-na.ssl-images-amazon.com/image...,"Arts, Crafts & Sewing",,">#76,268 in Arts, Crafts & Sewing (See Top 100...",,,,Wiley Publishers Crocheting For Dummies Revised,"[1118661516, 0470045558, 156477998X, B00QF75NF..."
11,"[B074XN2B21, B078S2GVQT, B01G2DLWZU, 149500914...",,0692218645,Troubadour Music Company,"[Musical Instruments, Instrument Accessories, ...","October 27, 2014",[The Ukulele Chord Changer fits soprano ukulel...,,"[Chords include C7, C, G7, G, A7, F, D7, and D...",,[https://images-na.ssl-images-amazon.com/image...,Musical Instruments,,">#26,851 in Musical Instruments (See Top 100 i...","[{'asin': '', 'href': '', 'name': 'Ukulele Cho...",,,Ukulele Chord Changer and Songbook Set,"[B074XN2B21, B078S2GVQT, B01G2DLWZU, 149500914..."


In [176]:
aprioridf.shape[0]

17125

In [182]:
#also buy column is a. list of items alonh with self bought item,
transactions=aprioridf.also_buy_appended.to_list()

In [184]:
#generate list of tuples having items bought together 
nested_lst_of_transactions = [tuple(l) for l in transactions]
nested_lst_of_transactions

[('B00NP8GYVS', 'B00NP80XMO', 'B00NP8M098', '0000989983'),
 ('B00NP8GYVS',
  'B00NP80XMO',
  'B01MY48HK5',
  'B00AZZ1AJK',
  'B0149IWDS0',
  'B0042SR4IW',
  'B001NZB2D0',
  'B076B9TY5W',
  'B00NP8M098',
  'B07CM6H5S7',
  'B003RQH5OI',
  'B01881NN10',
  'B00YXP43LU',
  'B0002F4UYU',
  'B0002EB5YO',
  'B000EEJ6P2',
  'B004MAPA22',
  'B00OPGA0LY',
  'B0146137RG',
  'B0002IBTXW',
  'B001O5VZ0E',
  'B06XKM3WK1',
  'B002U51K1G',
  'B005MA1I2W',
  'B000WIT8BM',
  'B000EEL8A8',
  'B00X1PGN5W',
  'B000EEL4YI',
  'B01N6BKF3F',
  'B01HJEHEH6',
  'B0002EQT68',
  'B0000775G0',
  'B07BGSQLKB',
  'B015Y18ULU',
  'B0033PQVOI',
  'B01F6MLDOS',
  'B00YXP4B3U',
  'B00HEXV618',
  'B01N6PJWWW',
  'B0002Q1XM6',
  'B06WP475Y4',
  'B007RNTLQM',
  'B008C1TYWE',
  'B0002F51YS',
  'B018N41C9C',
  'B00PB34DSG',
  'B0002F5CQK',
  'B001DZVSYS',
  'B00HE81YGU',
  'B001D4KVWY',
  'B002JK0L1C',
  'B0009V84I0',
  'B077CZCP6V',
  'B007RA3GPM',
  'B000U1UOP0',
  'B079JJZXJ4',
  'B00N2MIE6A',
  'B077TP3QV3',
  'B007L3Z1RQ

In [191]:
from apyori import apriori

In [223]:
100/aprioridf.shape[0]

0.00583941605839416

In [262]:
#using apriori to generate datasets 
association_rules = apriori(transactions, min_support=0.0058, min_confidence=0.70, min_lift=3, min_length=3)
association_results = list(association_rules)

In [263]:
print(len(association_results))

259


In [264]:
print(association_results[0])
a=association_results[0].items
print(a)
#get frequently bought items sets

RelationRecord(items=frozenset({'B0002D0E88', 'B0002D0DVQ'}), support=0.006598540145985402, ordered_statistics=[OrderedStatistic(items_base=frozenset({'B0002D0E88'}), items_add=frozenset({'B0002D0DVQ'}), confidence=0.7243589743589743, lift=55.37789033882784)])
frozenset({'B0002D0E88', 'B0002D0DVQ'})


In [307]:
# for our user in consideration we see what items has he most recently bought and suggest bundles where these items occur together.
#enter user id for which bundle to be genearted 
#and run below code

userid= "A265SU6RW4ZAPY"
userid2="A215GPS2IFRI7G"

subset_ratings_user=AppliancesDf.copy()
subset_ratings_train_userspecific=subset_ratings_user[subset_ratings_user.reviewerID==userid]
subset_ratings_train_userspecific=subset_ratings_train_userspecific[subset_ratings_train_userspecific['verified']==True]
subset_ratings_train_userspecific['reviewDate']=pd.to_datetime(subset_ratings_train_userspecific['reviewTime'])


subset_ratings_train_userspecific.shape

(41, 13)

In [308]:
subset_ratings_train_userspecific=subset_ratings_train_userspecific.sort_values(by=['reviewDate'], ascending=False)

In [309]:
subset_ratings_train_userspecific.head()

,asin,image,overall,reviewText,reviewTime,reviewerID,reviewerName,style,summary,unixReviewTime,verified,vote,reviewDate
1501564,B01EJ0N5FS,NaN,5.0,Works fine.,"09 14, 2018",A265SU6RW4ZAPY,3Gr8KidsDad,NaN,Works fine.,1536883200,True,NaN,2018-09-14
1454675,B011MBAUQY,NaN,5.0,"Very pleased with this product. Heavy duty, q...","03 19, 2018",A265SU6RW4ZAPY,3Gr8KidsDad,NaN,"Very pleased with this product. Heavy duty, q...",1521417600,True,NaN,2018-03-19
1433933,B00URFIYC4,NaN,5.0,Very pleased with Donner's transmitters and re...,"03 19, 2018",A265SU6RW4ZAPY,3Gr8KidsDad,{'Color:': ' Transmitter'},Very pleased with Donner's transmitters and re...,1521417600,True,NaN,2018-03-19
1421445,B00RC44JZO,NaN,5.0,"Sturdy, thin clamps!","03 19, 2018",A265SU6RW4ZAPY,3Gr8KidsDad,{'Color:': ' 0'},"Sturdy, thin clamps!",1521417600,True,NaN,2018-03-19
1110751,B000960KNW,NaN,5.0,Quality strap locks!,"03 19, 2018",A265SU6RW4ZAPY,3Gr8KidsDad,NaN,Quality strap locks!,1521417600,True,NaN,2018-03-19


In [310]:
mostrecentitem=subset_ratings_train_userspecific.asin.unique()
#mostrecentitem=mostrecentitem[0]

In [311]:
mostrecentitem

array(['B01EJ0N5FS', 'B011MBAUQY', 'B00URFIYC4', 'B00RC44JZO',
       'B000960KNW', 'B0002E1H9W', 'B018S1XCSE', 'B0192Y5YXM',
       'B009XJU28O', 'B003ZJSHM6', 'B001P33HVK', 'B000CD3QYW',
       'B00076R384', 'B01FF4LGWQ', 'B0192Y7UPC', 'B0002V8R5M',
       'B000068O29', 'B00ZU86PEC', 'B00KO8W060', 'B00G6IKCQW',
       'B003ZAOQBQ', 'B00FKY86V6', 'B0098DFNY8', 'B005FO1WBC',
       'B00GXNBI18', 'B01BU6W6RW', 'B0064RU0W6', 'B00OGZVTZA',
       'B016ZMDFGW', 'B01DYPZ598', 'B001QJGYIG', 'B00E58SW3U',
       'B0010CZ8GS', 'B00J7EEAOC', 'B00KWIM2FG', 'B0092LWHI6',
       'B000NE4UN6', 'B000ARH3M6', 'B01BXR6O2Q', 'B00E411NG6'],
      dtype=object)

In [312]:
#now, identify which frequent sets has this item from the set of generated results.
frequentdf = []
for item in association_results:
    
    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    #print("items is, " ,items)
    value0 = str(items[0])
    value1 = str(items[1])
    
    rows = (value0, value1)
    frequentdf.append(rows)
    

print(frequentdf)

[('B0002D0E88', 'B0002D0DVQ'), ('B0002D0DVQ', 'B00GXNZP80'), ('B0002D0E88', 'B0002D0H8U'), ('B0002D0E88', 'B0002E2OSU'), ('B0002D0E88', 'B00GXNZP80'), ('B0002D0H8U', 'B001APXNE4'), ('B0002D0H8U', 'B00GXNZP80'), ('B0002E1O2C', 'B01H74YV56'), ('B0002KZE7C', 'B0002E2GMY'), ('B00721BB44', 'B0002E2GMY'), ('B01N3LOKPG', 'B0002E2GMY'), ('B0002E2OSU', 'B0002E54FK'), ('B0002E2OSU', 'B00GXNZP80'), ('B0002E2TVM', 'B00GXNZP80'), ('B0002E54FK', 'B00GXNZP80'), ('B0002F73TE', 'B00GXNZP80'), ('B01DYPZ598', 'B0002KZDN2'), ('B0002KZE7C', 'B0002KZIT6'), ('B0002KZE7C', 'B000WN4J9S'), ('B00721BB44', 'B0002KZE7C'), ('B0002KZE7C', 'B00C5B20QE'), ('B00G964BUY', 'B0002KZE7C'), ('B00KRQP6BK', 'B0002KZE7C'), ('B0002KZE7C', 'B01N3LOKPG'), ('B0006NDF8A', 'B001L8PIFW'), ('B0006NDF8A', 'B001PGXHXA'), ('B0006NDF8A', 'B00DSSOVJW'), ('B0006NDF8A', 'B018FCZKR2'), ('B001APXNE4', 'B00GXNZP80'), ('B005PNXT6E', 'B001L8PIFW'), ('B001SEQQDC', 'B00GXNZP80'), ('B002FEGREW', 'B001SN7VGO'), ('B002FEH6AQ', 'B001SN7VGO'), ('B002FEG

In [313]:
finallist=[]
for j in mostrecentitem:
    
    for i in frequentdf:
        if (j in i):
            finallist.append(i)
finallist

[('B01DYPZ598', 'B0002KZDN2'),
 ('B01DYPZ598', 'B00721BB44'),
 ('B01DYPZ598', 'B00C5B20QE'),
 ('B01DYPZ598', 'B01N3LOKPG'),
 ('B01DYPZ598', 'B0002E2G2E'),
 ('B01DYPZ598', 'B0002E2G2E'),
 ('B01DYPZ598', 'B0002E2G2E'),
 ('B01I1XNY1E', 'B01DYPZ598'),
 ('B01DYPZ598', 'B0002KZE7C'),
 ('B01DYPZ598', 'B000L6GD04'),
 ('B01DYPZ598', 'B0002E2GMY'),
 ('B01DYPZ598', 'B0002E2GMY'),
 ('B01DYPZ598', 'B00CAV0TRQ'),
 ('B01DYPZ598', 'B0002E2GMY'),
 ('B01I1XNY1E', 'B01DYPZ598'),
 ('B01DYPZ598', 'B01N3LOKPG'),
 ('B01DYPZ598', 'B000L6GD04'),
 ('B01DYPZ598', 'B0002KZE7C'),
 ('B01DYPZ598', 'B0002KZE7C'),
 ('B01DYPZ598', 'B0002KZE7C'),
 ('B01I1XNY1E', 'B01DYPZ598'),
 ('B01DYPZ598', 'B0002KZE7C'),
 ('B01DYPZ598', 'B0002KZE7C'),
 ('B01DYPZ598', 'B000L6GD04'),
 ('B01DYPZ598', 'B000L6GD04'),
 ('B01DYPZ598', 'B000L6GD04'),
 ('B01I1XNY1E', 'B01DYPZ598'),
 ('B01DYPZ598', 'B000L6GD04'),
 ('B01DYPZ598', 'B00CAV0TRQ'),
 ('B01DYPZ598', 'B000WN4J9S'),
 ('B01I1XNY1E', 'B01DYPZ598'),
 ('B01DYPZ598', 'B01N3LOKPG'),
 ('B01DY

In [314]:
labels = ['item 1','item 2']
itembundle_suggestion = pd.DataFrame.from_records(finallist, columns = labels)
itembundle_suggestion.shape

(61, 2)

In [315]:
itembundle_suggestion=itembundle_suggestion.drop_duplicates()
itembundle_suggestion.shape

(12, 2)

In [316]:
itembundle_suggestion

,item 1,item 2
0,B01DYPZ598,B0002KZDN2
1,B01DYPZ598,B00721BB44
2,B01DYPZ598,B00C5B20QE
3,B01DYPZ598,B01N3LOKPG
4,B01DYPZ598,B0002E2G2E
7,B01I1XNY1E,B01DYPZ598
8,B01DYPZ598,B0002KZE7C
9,B01DYPZ598,B000L6GD04
10,B01DYPZ598,B0002E2GMY
12,B01DYPZ598,B00CAV0TRQ


In [317]:
itemdict=AppliancesmetaDf_cleaned[['asin','title']]
itemdict=itemdict.drop_duplicates()
itemdict=itemdict.set_index('asin').T
itemdict=itemdict.to_dict('list')

In [318]:
#user1
itembundle_suggestion['item 1 name']=itembundle_suggestion['item 1'].map(itemdict)
itembundle_suggestion['item 2 name']=itembundle_suggestion['item 2'].map(itemdict)
itembundle_suggestion

,item 1,item 2,item 1 name,item 2 name
0,B01DYPZ598,B0002KZDN2,[30 Feet (10-white/10-black/10-yellow) Gavitt ...,[Fender 5-way Selector Switch]
1,B01DYPZ598,B00721BB44,[30 Feet (10-white/10-black/10-yellow) Gavitt ...,[Switchcraft 11 Guitar Input Mono Phone Panel ...
2,B01DYPZ598,B00C5B20QE,[30 Feet (10-white/10-black/10-yellow) Gavitt ...,[Switchcraft 11 Mono Female 1/4-Inch Jack with...
3,B01DYPZ598,B01N3LOKPG,[30 Feet (10-white/10-black/10-yellow) Gavitt ...,NaN
4,B01DYPZ598,B0002E2G2E,[30 Feet (10-white/10-black/10-yellow) Gavitt ...,[Fender 3-Position Vintage-Style Stratocaster/...
7,B01I1XNY1E,B01DYPZ598,NaN,[30 Feet (10-white/10-black/10-yellow) Gavitt ...
8,B01DYPZ598,B0002KZE7C,[30 Feet (10-white/10-black/10-yellow) Gavitt ...,[Fender Pickguard Screws]
9,B01DYPZ598,B000L6GD04,[30 Feet (10-white/10-black/10-yellow) Gavitt ...,[Fender Locking Tuners - Polished Chrome]
10,B01DYPZ598,B0002E2GMY,[30 Feet (10-white/10-black/10-yellow) Gavitt ...,[Fender American Series String Guides]
12,B01DYPZ598,B00CAV0TRQ,[30 Feet (10-white/10-black/10-yellow) Gavitt ...,NaN


In [319]:
#user1
bundles=[]
item2list1=itembundle_suggestion['item 2 name'].to_list()
#user 1
for i in range(0,len(item2list1),3):
    bundles.append([item2list1[i],item2list1[i+1],item2list1[i+2]])



In [320]:
bundles
#for user 1


[[['Fender 5-way Selector Switch'],
  ['Switchcraft 11 Guitar Input Mono Phone Panel Jack'],
  ['Switchcraft 11 Mono Female 1/4-Inch Jack with Nut and Washer, Nickel Finish']],
 [nan,
  ['Fender 3-Position Vintage-Style Stratocaster/Telecaster Pickup Selector Switch'],
  ['30 Feet (10-white/10-black/10-yellow) Gavitt Cloth-covered Pre-tinned 7-strand Pushback 22awg Vintage-style Guitar Wire']],
 [['Fender Pickguard Screws'],
  ['Fender Locking Tuners - Polished Chrome'],
  ['Fender American Series String Guides']],
 [nan,
  ['Fender F Neckplate Chrome'],
  ['2x (Pair) .022uf/400v Orange Drop Film Capacitors']]]

In [301]:
#user 2
itembundle_suggestion['item 1 name']=itembundle_suggestion['item 1'].map(itemdict)
itembundle_suggestion['item 2 name']=itembundle_suggestion['item 2'].map(itemdict)
itembundle_suggestion

,item 1,item 2,item 1 name,item 2 name
0,B00G964BUY,B0002KZE7C,[ElectroSocket Jackplate for Tele Chrome],[Fender Pickguard Screws]
2,B00G964BUY,B000L6GD04,[ElectroSocket Jackplate for Tele Chrome],[Fender Locking Tuners - Polished Chrome]
3,B00G964BUY,B0002E2GMY,[ElectroSocket Jackplate for Tele Chrome],[Fender American Series String Guides]


In [302]:
#user 2
itembundle_suggestion 
# item 2 names are the items that can be plugged with item 1 which the user has bought.

,item 1,item 2,item 1 name,item 2 name
0,B00G964BUY,B0002KZE7C,[ElectroSocket Jackplate for Tele Chrome],[Fender Pickguard Screws]
2,B00G964BUY,B000L6GD04,[ElectroSocket Jackplate for Tele Chrome],[Fender Locking Tuners - Polished Chrome]
3,B00G964BUY,B0002E2GMY,[ElectroSocket Jackplate for Tele Chrome],[Fender American Series String Guides]


In [303]:
#user2
bundles=[]
item2list=itembundle_suggestion['item 2 name'].to_list()

In [304]:
#user 2
for i in range(0,len(item2list),3):
    bundles.append([item2list[i],item2list[i+1],item2list[i+2]])


In [306]:
#user2
bundles[0]

[['Fender Pickguard Screws'],
 ['Fender Locking Tuners - Polished Chrome'],
 ['Fender American Series String Guides']]